<a href="https://colab.research.google.com/github/Praveen76/Introduction-to-Huggingface/blob/main/Hf_Tokenizer_Models_and_Ouputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Huggingface Tokenizer, Models and Model Ouputs**

### Objectives:

At the end of the experiment you will be able to understand  :

1. tokenizer class in HF
2. models in HF
3. model output and interpret
4. built in pipline in HF and use for different task

### **Tokenizer**
A tokenizer is in charge of preparing the inputs for a model. The Transformder library contains tokenizers for all the models i.e. tokenizer rules are specific to models and differs model by model.

But there is a universal interface so that you don't have to worry about picking the right class. Specifically, there is a class called **`AutoTokenizer`**, where you can pass in a model checkpoints just like in  pipelines. This will automatically give you back the correct tokenized objects.

So for example, if your model checkpoint is based on BERT, you will get back a tokenized object that has all the right component in it as required by BERT model.

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint='bert-base-cased' # Different Bert models are there.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
tokenizer # Notice the output from tokenizer object

#### Tokenizing any text:

Note the result form below code, it is a dictionary with several keys. The key input IDs corresponds to the integer IDs of the tokens that have gotten back from tokenizing the input. The input IDs will always be present because you will always want to convert your text into token IDs. The other keys are attention, mask and token type IDs.

Note that these keys can be specific to the type of model. For instance, token type IDs will show up for BERT, but not to DistillBert.

In [ ]:
tokenizer("Hello Dost")

{'input_ids': [101, 8667, 2091, 2050, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

#### Using tokenizer object

In [ ]:
tokens= tokenizer.tokenize("Hello Dost")
tokens

['Hello', 'Do', '##st']

#### Tokens into token Ids

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[8667, 2091, 2050]

#### Above two steps can be done in one go by encoding

In [ ]:
ids_n = tokenizer.encode("Hello Dost")
ids_n

[101, 8667, 2091, 2050, 102]

**Note** above, 5 ids are because the input has been converted to [CLS],  'Hello', 'Do', '##st',  [SEP]**

In [ ]:
tokenizer.convert_ids_to_tokens(ids_n)

['[CLS]', 'Hello', 'Do', '##st', '[SEP]']

In [ ]:
tokenizer.decode(ids_n) # Gives the single string with tokens joined back together

'[CLS] Hello Dost [SEP]'

#### Getting tensor as an output

The output from tokenizer that we just saw, is a dictionary containing values, which were all lists. But PyTorch doesn't take this as input.

Instead, PyTorch models process torch Tensors in order to get the values back as Tensors. We set the argument `return_tensors` to  string `pt`.

For TensorFlo set string as `tf` and for numpy as `np`you can use the string TAF or if you just want an empire raise, you can pass an NP.



In [ ]:
tokenizer("Hello Dost",return_tensors='pt') # tf, np

{'input_ids': tensor([[ 101, 8667, 2091, 2050,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

#### Multiple inputs : Need to add two more parameters : `padding` and `truncation`

In [ ]:
data = [ "Hello where are you going?","Going to home.", "I like playing cricket."]

In [ ]:
model_inputs=tokenizer(data,padding=True,truncation=True,return_tensors='pt')
model_inputs

{'input_ids': tensor([[  101,  8667,  1187,  1132,  1128,  1280,   136,   102],
        [  101, 11099,  1106,  1313,   119,   102,     0,     0],
        [  101,   146,  1176,  1773,  5428,   119,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]])}

Purpose of the attention mask: Essentially, this tells the model where it should bother to pay attention to. For any tokens where the attention mask is zero, the model will ignore those tokens and it will not be possible to use them to compute the model output, which is  as intended.

In summary to ensure that the batch of data can be fed as input into to the PyTorch model, We need to specify the padding argument, the truncation argument and the return tensor argument. Also note that PyTorch is the default for hugging face and currently the most flexible.

### **Model**

We are using BERT model for text classification, through  `AutoModelForSequenceClassification` class since it it more flexible. We can create a BERT Specific model In order to load a pre-trained BERT model, we simply call the function from Pre-Trained. Just like we did with the tokenizer.

**Note that the checkpoint we pass in must match the checkpoint we passed in for the tokenizer so that we get the right tokenize for the model**.

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
checkpoint = checkpoint='bert-base-cased' # cased - both capital and lower case| uncased - only lower case
model= AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Making Predictions

In [ ]:
model_inputs = tokenizer(data, padding = True, return_tensors = 'pt')
model_inputs


{'input_ids': tensor([[  101,  8667,  1187,  1132,  1128,  1280,   136,   102],
        [  101, 11099,  1106,  1313,   119,   102,     0,     0],
        [  101,   146,  1176,  1773,  5428,   119,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]])}

In [ ]:
model_inputs['input_ids']

tensor([[  101,  8667,  1187,  1132,  1128,  1280,   136,   102],
        [  101, 11099,  1106,  1313,   119,   102,     0,     0],
        [  101,   146,  1176,  1773,  5428,   119,   102,     0]])

In [ ]:
model_inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]])

In [ ]:
outputs = model(**model_inputs) ## ** key value pair, named arguments

In [ ]:
outputs # These logits are useless as final layers are not tuned and by default it assumes binary classification

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0513, -0.7316],
        [ 0.0926, -0.7506],
        [ 0.0666, -0.7974]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

### **Model Outputs**
* If you pass in N documents, you will get back NxK, as an output where k is for representing number of classes.
* If you pass in a single document, you will get back a K-sized output.
* The outputs are logits i.e. value before applying softmax.
* To get class prediction, just take the argmax.


In [ ]:
model_n= AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model_n

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
outputs_n=model_n(**model_inputs)
outputs_n

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2327, -0.6322,  0.2688],
        [-0.3049, -0.5836,  0.2060],
        [-0.2424, -0.6952,  0.2346]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
outputs.logits # Method1

tensor([[ 0.0513, -0.7316],
        [ 0.0926, -0.7506],
        [ 0.0666, -0.7974]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs['logits'] # Method2

tensor([[ 0.0513, -0.7316],
        [ 0.0926, -0.7506],
        [ 0.0666, -0.7974]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs[0] # Method3 --> Not recommended

tensor([[ 0.0513, -0.7316],
        [ 0.0926, -0.7506],
        [ 0.0666, -0.7974]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs.logits.detach().cpu().numpy() # convert into numpy array

array([[ 0.05132937, -0.73155737],
       [ 0.09263515, -0.7505686 ],
       [ 0.06659517, -0.79740906]], dtype=float32)

### **Built in Pipeline**

#### **Example-1: Pipeline-Sentiment Analysis**

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I ate Biryani in HYD not so good.", "I like the book by Geron in ML but it is in Tensorflow"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'NEGATIVE', 'score': 0.9992625117301941},
 {'label': 'NEGATIVE', 'score': 0.5498270392417908}]

In [ ]:
type(sentiment_pipeline)

transformers.pipelines.text_classification.TextClassificationPipeline

#### **Example-2: Pipeline-Question Answering**

In [ ]:
# The task "question-answering" will return a QuestionAnsweringPipeline object
question_answer = pipeline(task="question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
context = """
Tea is an aromatic beverage prepared by pouring hot or boiling water over cured or fresh leaves of Camellia sinensis,
an evergreen shrub native to China and East Asia. After water, it is the most widely consumed drink in the world.
There are many different types of tea; some, like Chinese greens and Darjeeling, have a cooling, slightly bitter,
and astringent flavour, while others have vastly different profiles that include sweet, nutty, floral, or grassy
notes. Tea has a stimulating effect in humans primarily due to its caffeine content.

The tea plant originated in the region encompassing today's Southwest China, Tibet, north Myanmar and Northeast India,
where it was used as a medicinal drink by various ethnic groups. An early credible record of tea drinking dates to
the 3rd century AD, in a medical text written by Hua Tuo. It was popularised as a recreational drink during the
Chinese Tang dynasty, and tea drinking spread to other East Asian countries. Portuguese priests and merchants
introduced it to Europe during the 16th century. During the 17th century, drinking tea became fashionable among the
English, who started to plant tea on a large scale in India.

The term herbal tea refers to drinks not made from Camellia sinensis: infusions of fruit, leaves, or other plant
parts, such as steeps of rosehip, chamomile, or rooibos. These may be called tisanes or herbal infusions to prevent
confusion with 'tea' made from the tea plant."""

In [ ]:
result = question_answer(question="Where is tea native to?", context=context)
print(result['answer'])

China and East Asia


In [ ]:
questions = ["Where is tea native to?",
             "When was tea discovered?",
             "What is the species name for tea?"]

results = question_answer(question=questions, context=context)

for q, r in zip(questions, results):
    print(q, "\n>> " + r['answer'])

Where is tea native to? 
>> China and East Asia
When was tea discovered? 
>> 3rd century AD
What is the species name for tea? 
>> Camellia sinensis


#### [Pipeline Examples For almost all Task](https://www.kdnuggets.com/2023/02/simple-nlp-pipelines-huggingface-transformers.html)